In [ ]:
# ==================== 1. INSTALACIÓN LIMPIA Y COMPATIBLE ====================
print("🔧 Realizando instalación limpia...")

# Desinstalar versiones conflictivas
!pip uninstall -y torch torchvision torchaudio transformers sentence-transformers gradio -q

# Instalar PyTorch compatible con Google Colab (CUDA 11.8)
!pip install torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 --index-url https://download.pytorch.org/whl/cu118 -q

# Instalar transformers y dependencias principales
!pip install transformers sentence-transformers -q
!pip install chromadb==0.4.22 pypdf PyPDF2 pdfplumber pymupdf Pillow -q
!pip install gradio==4.12.0 accelerate bitsandbytes -q

print("✅ Instalación completada. Reiniciando entorno...")

# Reiniciar el kernel (IMPORTANTE)
import os
os._exit(00)

🔧 Realizando instalación limpia...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.0 requires transformers, which is not installed.


In [ ]:
# ==================== 2. MONTAR GOOGLE DRIVE PARA PERSISTENCIA ====================
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

# Crear estructura de carpetas
DRIVE_PATH = "/content/drive/MyDrive/RAG_Hispanidad"
VECTOR_DB_PATH = f"{DRIVE_PATH}/vector_db"
PDF_STORAGE_PATH = f"{DRIVE_PATH}/pdf_storage"

os.makedirs(DRIVE_PATH, exist_ok=True)
os.makedirs(VECTOR_DB_PATH, exist_ok=True)
os.makedirs(PDF_STORAGE_PATH, exist_ok=True)

print(f"📁 Drive montado en: {DRIVE_PATH}")
print(f"🗄️  Base de vectores: {VECTOR_DB_PATH}")
print(f"📄 PDFs almacenados: {PDF_STORAGE_PATH}")

Mounted at /content/drive
📁 Drive montado en: /content/drive/MyDrive/RAG_Hispanidad
🗄️  Base de vectores: /content/drive/MyDrive/RAG_Hispanidad/vector_db
📄 PDFs almacenados: /content/drive/MyDrive/RAG_Hispanidad/pdf_storage


In [ ]:
# ==================== 3. IMPORTAR LIBRERÍAS ====================
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import json
import hashlib
import time
from datetime import datetime
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')
import fitz  # PyMuPDF
import pdfplumber
from PyPDF2 import PdfReader
import numpy as np

# Configurar GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Usando: {device}")
if torch.cuda.is_available():
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")

# ==================== 4. CONFIGURACIÓN ====================
MODEL_NAME = "BSC-LT/salamandra-2b-instruct"
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

🚀 Usando: cuda
📊 GPU: Tesla T4


In [ ]:
# ==================== 5. EXTRACTOR DE PDF INTELIGENTE ====================
class SmartPDFExtractor:
    def __init__(self):
        print("📄 Extractor de PDFs inicializado")

    def extract_with_pymupdf(self, pdf_path: str):
        text = ""
        metadata = {}

        try:
            doc = fitz.open(pdf_path)
            metadata = {
                'pages': len(doc),
                'author': doc.metadata.get('author', ''),
                'title': doc.metadata.get('title', ''),
                'subject': doc.metadata.get('subject', '')
            }

            for page_num in range(len(doc)):
                page = doc[page_num]
                page_text = page.get_text()
                if page_text.strip():
                    text += f"\n\n--- Página {page_num + 1} ---\n{page_text}"

            doc.close()
            return text.strip(), metadata

        except Exception as e:
            print(f"⚠️ PyMuPDF falló: {e}")
            return "", {}

    def extract_with_pdfplumber(self, pdf_path: str):
        text = ""

        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page_num, page in enumerate(pdf.pages):
                    page_text = page.extract_text()
                    if page_text:
                        text += f"\n\n--- Página {page_num + 1} ---\n{page_text}"

            return text.strip(), {'pages': len(pdf.pages)}

        except Exception as e:
            print(f"⚠️ pdfplumber falló: {e}")
            return "", {}

    def extract_text(self, pdf_path: str):
        print(f"📖 Extrayendo: {os.path.basename(pdf_path)}")

        # Intentar PyMuPDF primero
        text1, meta1 = self.extract_with_pymupdf(pdf_path)
        if text1 and len(text1) > 100:
            text_len = len(text1)
            print(f"✅ PyMuPDF: {text_len:,} caracteres, {meta1.get('pages', '?')} páginas")

            avg_words = len(text1.split()) / max(1, meta1.get('pages', 1))
            quality = "baja" if avg_words < 50 else "media" if avg_words < 200 else "alta"

            return text1, meta1, quality

        # Fallback a pdfplumber
        text2, meta2 = self.extract_with_pdfplumber(pdf_path)
        if text2 and len(text2) > 100:
            text_len = len(text2)
            print(f"✅ pdfplumber: {text_len:,} caracteres, {meta2.get('pages', '?')} páginas")

            avg_words = len(text2.split()) / max(1, meta2.get('pages', 1))
            quality = "baja" if avg_words < 50 else "media" if avg_words < 200 else "alta"

            return text2, meta2, quality

        print(f"❌ Todos los métodos fallaron")
        return "", {}, "error"

In [ ]:
# ==================== 6. GESTOR DE PDFS ====================
class PDFManager:
    def __init__(self, storage_path: str):
        self.storage_path = storage_path
        self.extractor = SmartPDFExtractor()
        self.processed_log = {}
        self.load_processed_log()

    def load_processed_log(self):
        log_path = os.path.join(self.storage_path, "processed_log.json")
        if os.path.exists(log_path):
            try:
                with open(log_path, 'r', encoding='utf-8') as f:
                    self.processed_log = json.load(f)
                print(f"📋 Log cargado: {len(self.processed_log)} PDFs")
            except:
                self.processed_log = {}

    def save_processed_log(self):
        log_path = os.path.join(self.storage_path, "processed_log.json")
        with open(log_path, 'w', encoding='utf-8') as f:
            json.dump(self.processed_log, f, indent=2, ensure_ascii=False)

    def save_pdf_file(self, pdf_bytes: bytes, original_filename: str) -> str:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_name = ''.join(c for c in original_filename if c.isalnum() or c in ' ._-')
        unique_name = f"{timestamp}_{safe_name}"

        save_path = os.path.join(self.storage_path, unique_name)

        with open(save_path, 'wb') as f:
            f.write(pdf_bytes)

        return save_path

    def process_pdf(self, pdf_bytes: bytes, original_filename: str) -> Dict:
        print(f"\n{'='*50}")
        print(f"📤 PROCESANDO: {original_filename}")
        print(f"{'='*50}")

        temp_path = os.path.join(self.storage_path, f"temp_{int(time.time())}.pdf")
        with open(temp_path, 'wb') as f:
            f.write(pdf_bytes)

        try:
            text, metadata, quality = self.extractor.extract_text(temp_path)

            if not text or len(text) < 100:
                os.remove(temp_path)
                return {
                    'success': False,
                    'error': 'PDF sin texto extraíble',
                    'filename': original_filename
                }

            content_hash = hashlib.md5(text.encode()).hexdigest()

            for pdf_info in self.processed_log.values():
                if pdf_info.get('content_hash') == content_hash:
                    print(f"⏭️  PDF ya procesado")
                    os.remove(temp_path)
                    return {
                        'success': False,
                        'error': 'PDF duplicado',
                        'filename': original_filename
                    }

            final_path = self.save_pdf_file(pdf_bytes, original_filename)

            pdf_info = {
                'filename': original_filename,
                'stored_name': os.path.basename(final_path),
                'path': final_path,
                'content_hash': content_hash,
                'text_length': len(text),
                'pages': metadata.get('pages', '?'),
                'quality': quality,
                'author': metadata.get('author', ''),
                'title': metadata.get('title', ''),
                'processed_date': datetime.now().isoformat(),
                'chunks_generated': 0,
                'status': 'processed'
            }

            pdf_id = f"pdf_{len(self.processed_log) + 1:04d}"
            self.processed_log[pdf_id] = pdf_info
            self.save_processed_log()

            print(f"✅ PDF procesado exitosamente")
            print(f"   📄 Páginas: {pdf_info['pages']}")
            print(f"   📊 Caracteres: {pdf_info['text_length']:,}")

            return {
                'success': True,
                'pdf_id': pdf_id,
                'text': text,
                'metadata': pdf_info,
                'original_filename': original_filename
            }

        except Exception as e:
            print(f"❌ Error: {e}")
            return {
                'success': False,
                'error': str(e),
                'filename': original_filename
            }
        finally:
            if os.path.exists(temp_path):
                os.remove(temp_path)

    def get_pdf_stats(self) -> Dict:
        total_pdfs = len(self.processed_log)
        total_pages = sum(info.get('pages', 0) for info in self.processed_log.values() if isinstance(info.get('pages'), int))
        total_chars = sum(info.get('text_length', 0) for info in self.processed_log.values())

        quality_counts = {}
        for info in self.processed_log.values():
            quality = info.get('quality', 'desconocida')
            quality_counts[quality] = quality_counts.get(quality, 0) + 1

        return {
            'total_pdfs': total_pdfs,
            'total_pages': total_pages,
            'total_chars': f"{total_chars:,}",
            'quality_distribution': quality_counts,
            'last_update': datetime.now().strftime("%Y-%m-%d %H:%M")
        }

In [ ]:
# ==================== 7. VECTOR STORE PERSISTENTE ====================
class PersistentVectorStore:
    def __init__(self, persist_path: str):
        self.persist_path = persist_path

        self.client = chromadb.PersistentClient(
            path=persist_path,
            settings=Settings(anonymized_telemetry=False)
        )

        try:
            self.collection = self.client.get_collection(name="hispanidad_docs")
            print(f"📚 Colección cargada: {self.collection.name} ({self.collection.count()} documentos)")
        except:
            self.collection = self.client.create_collection(
                name="hispanidad_docs",
                metadata={"description": "Documentos históricos hispánicos"}
            )
            print("🆕 Nueva colección creada")

    def add_pdf_chunks(self, pdf_id: str, text: str, pdf_metadata: Dict) -> int:
        paragraphs = [p.strip() for p in text.split('\n\n') if p.strip() and len(p.strip()) > 100]

        chunks = []
        metadatas = []
        ids = []

        current_chunk = ""
        chunk_num = 0

        for para in paragraphs:
            if len(current_chunk) + len(para) < 1500:
                if current_chunk:
                    current_chunk += "\n\n" + para
                else:
                    current_chunk = para
            else:
                if current_chunk:
                    chunk_metadata = {
                        'pdf_id': pdf_id,
                        'pdf_title': pdf_metadata.get('title', pdf_metadata.get('filename', '')),
                        'pdf_author': pdf_metadata.get('author', ''),
                        'pdf_pages': pdf_metadata.get('pages', 0),
                        'chunk_num': chunk_num,
                        'total_chunks': 0,
                        'type': 'historia_hispanica',
                        'source': 'PDF',
                        'quality': pdf_metadata.get('quality', 'media')
                    }

                    chunks.append(current_chunk)
                    metadatas.append(chunk_metadata)
                    ids.append(f"{pdf_id}_chunk_{chunk_num}")

                    chunk_num += 1
                    current_chunk = para

        if current_chunk and len(chunks) < 1000:
            chunk_metadata = {
                'pdf_id': pdf_id,
                'pdf_title': pdf_metadata.get('title', pdf_metadata.get('filename', '')),
                'pdf_author': pdf_metadata.get('author', ''),
                'pdf_pages': pdf_metadata.get('pages', 0),
                'chunk_num': chunk_num,
                'total_chunks': chunk_num + 1,
                'type': 'historia_hispanica',
                'source': 'PDF',
                'quality': pdf_metadata.get('quality', 'media')
            }

            chunks.append(current_chunk)
            metadatas.append(chunk_metadata)
            ids.append(f"{pdf_id}_chunk_{chunk_num}")
            chunk_num += 1

        for metadata in metadatas:
            metadata['total_chunks'] = chunk_num

        if chunks:
            self.collection.add(
                documents=chunks,
                metadatas=metadatas,
                ids=ids
            )
            print(f"   📝 Añadidos {len(chunks)} chunks")

        return len(chunks)

    def search(self, query: str, n_results: int = 4) -> List[Dict]:
        try:
            results = self.collection.query(
                query_texts=[query],
                n_results=n_results,
                include=["documents", "metadatas", "distances"]
            )

            formatted = []
            if results['documents']:
                for i, doc in enumerate(results['documents'][0]):
                    metadata = results['metadatas'][0][i]
                    formatted.append({
                        'text': doc[:800] + "..." if len(doc) > 800 else doc,
                        'metadata': metadata,
                        'score': 1 - (results['distances'][0][i] if results['distances'] else 0),
                        'pdf_title': metadata.get('pdf_title', 'Sin título')
                    })

            return formatted

        except Exception as e:
            print(f"❌ Error en búsqueda: {e}")
            return []

    def get_stats(self) -> Dict:
        try:
            count = self.collection.count()

            all_metas = self.collection.get(include=["metadatas"])
            pdf_ids = set()
            if all_metas['metadatas']:
                for meta in all_metas['metadatas']:
                    if meta and 'pdf_id' in meta:
                        pdf_ids.add(meta['pdf_id'])

            return {
                'total_chunks': count,
                'unique_pdfs': len(pdf_ids),
                'path': self.persist_path
            }
        except:
            return {'total_chunks': 0, 'unique_pdfs': 0}

In [ ]:
class PDFRAGSystem:
    def __init__(self):
        print("\n" + "="*60)
        print("🏛️  INICIALIZANDO SISTEMA RAG CON PDFs")
        print("="*60)

        self.pdf_manager = PDFManager(PDF_STORAGE_PATH)
        self.vector_store = PersistentVectorStore(VECTOR_DB_PATH)

        print("🧠 Cargando modelo salamandra-2b...")
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

        self.model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto"
        )

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        print("🔤 Cargando embeddings...")
        self.embedder = SentenceTransformer(EMBEDDING_MODEL)
        if torch.cuda.is_available():
            self.embedder = self.embedder.to(device)

        self.update_stats()

        print("\n✅ SISTEMA LISTO")
        print(f"📚 PDFs: {self.pdf_stats['total_pdfs']}")
        print(f"📝 Chunks: {self.vector_stats['total_chunks']}")

    def update_stats(self):
        self.pdf_stats = self.pdf_manager.get_pdf_stats()
        self.vector_stats = self.vector_store.get_stats()
        self.system_stats = {
            **self.pdf_stats,
            **self.vector_stats,
            'gpu': torch.cuda.is_available(),
            'model': MODEL_NAME,
            'last_update': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

    def upload_and_process_pdf(self, pdf_file, filename: str = None) -> Dict:
        if pdf_file is None:
            return {'success': False, 'error': 'No se seleccionó archivo'}

        try:
            if hasattr(pdf_file, 'read'):
                pdf_bytes = pdf_file.read()
                actual_filename = filename or getattr(pdf_file, 'name', 'documento.pdf')
            else:
                with open(pdf_file, 'rb') as f:
                    pdf_bytes = f.read()
                actual_filename = filename or os.path.basename(pdf_file)

            print(f"📤 Subiendo: {actual_filename}")

            result = self.pdf_manager.process_pdf(pdf_bytes, actual_filename)

            if not result['success']:
                return result

            pdf_id = result['pdf_id']
            text = result['text']
            metadata = result['metadata']

            print(f"📝 Generando chunks...")
            chunks_added = self.vector_store.add_pdf_chunks(pdf_id, text, metadata)

            if chunks_added > 0:
                self.pdf_manager.processed_log[pdf_id]['chunks_generated'] = chunks_added
                self.pdf_manager.processed_log[pdf_id]['status'] = 'indexed'
                self.pdf_manager.save_processed_log()

            self.update_stats()

            return {
                'success': True,
                'pdf_id': pdf_id,
                'filename': actual_filename,
                'chunks_added': chunks_added,
                'pages': metadata.get('pages', 0),
                'text_length': metadata.get('text_length', 0),
                'quality': metadata.get('quality', 'media'),
                'total_pdfs': self.system_stats['total_pdfs'],
                'total_chunks': self.system_stats['total_chunks']
            }

        except Exception as e:
            print(f"❌ Error: {e}")
            return {'success': False, 'error': str(e)}

    def search_documents(self, query: str, n_results: int = 4) -> List[Dict]:
        return self.vector_store.search(query, n_results)

    def generate_response(self, question: str, context_docs: List[Dict], max_chars: int = 2500) -> str:
        """Genera respuesta con parámetros optimizados para respuestas largas"""
        context_text = ""
        pdf_sources = {}

        # Limitar contexto para no sobrecargar el prompt
        for i, doc in enumerate(context_docs[:2]):  # Solo 2 documentos máximo
            source = doc.get('pdf_title', 'Documento PDF')
            if source not in pdf_sources:
                pdf_sources[source] = []
            pdf_sources[source].append(i + 1)

            # Tomar solo los primeros 400 caracteres de cada documento
            context_text += f"\n\n[📚 Referencia {i+1} - {source}]:\n{doc['text'][:400]}..."

        # Prompt más directivo
        prompt = f"""### INSTRUCCIÓN:
Eres un historiador experto en cultura hispánica. Responde de forma EXTENSA, DETALLADA y COMPLETA.
Usa al menos {max_chars//10} líneas en tu respuesta. NO TE CORTES a mitad de frase.

### CONTEXTO DOCUMENTAL:
{context_text}

### PREGUNTA DEL USUARIO:
{question}

### RESPUESTA EXTENSA Y DETALLADA (mínimo {max_chars//10} líneas):
"""

        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=1024  # Permitir más contexto
        ).to(self.model.device)

        with torch.no_grad():
            # PARÁMETROS OPTIMIZADOS PARA RESPUESTAS LARGAS
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=800,           # Más tokens para respuestas largas
                min_new_tokens=300,           # Mínimo de tokens a generar
                temperature=0.8,              # Un poco más creativo
                do_sample=True,               # Muestreo aleatorio
                top_p=0.95,                   # Nucleus sampling más amplio
                top_k=50,                     # Más opciones para elegir
                repetition_penalty=1.15,      # Penalizar repeticiones fuertemente
                no_repeat_ngram_size=4,       # Evitar n-gramas repetidos
                length_penalty=0.8,           # Penalizar menos las respuestas largas
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                num_return_sequences=1,
                use_cache=True
            )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extraer respuesta más agresivamente
        if "RESPUESTA EXTENSA Y DETALLADA" in response:
            response = response.split("RESPUESTA EXTENSA Y DETALLADA")[-1].strip()
        elif "### RESPUESTA" in response:
            response = response.split("### RESPUESTA")[-1].strip()

        # Limpiar marcadores residuales
        for marker in ["PREGUNTA DEL USUARIO:", "### PREGUNTA", "CONTEXTO DOCUMENTAL:", "###"]:
            if marker in response:
                response = response.split(marker)[0].strip()

        # Asegurar longitud mínima
        if len(response) < max_chars // 2:  # Si es menos de la mitad de lo solicitado
            print(f"⚠️  Respuesta muy corta ({len(response)} chars). Reintentando con más tokens...")
            # Reintentar con más tokens
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=1200,      # Mucho más tokens
                    temperature=0.85,
                    do_sample=True,
                    top_p=0.97,
                    repetition_penalty=1.1,
                    length_penalty=0.5,       # Favorecer respuestas aún más largas
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id
                )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            if "RESPUESTA" in response:
                response = response.split("RESPUESTA")[-1].strip()

        # Limitar tamaño final
        if len(response) > max_chars:
            # Intentar cortar en un punto razonable, no a mitad de frase
            if "." in response[max_chars-100:max_chars+100]:
                last_period = response[:max_chars+100].rfind(".")
                response = response[:last_period+1] + ".."
            else:
                response = response[:max_chars] + "..."

        # Añadir fuentes si hay
        if pdf_sources:
            sources_text = ", ".join([f"{source}" for source in pdf_sources.keys()])
            response += f"\n\n---\n📚 **Fuentes consultadas:** {sources_text}"

        print(f"📝 Longitud respuesta final: {len(response)} caracteres")
        return response

    def get_system_info(self) -> Dict:
        return self.system_stats

In [ ]:
# ==================== 9. INTERFAZ GRADIO CORREGIDA ====================
print("\n" + "="*60)
print("📤 INICIALIZANDO SISTEMA RAG...")
print("="*60)

# Inicializar sistema
pdf_rag = PDFRAGSystem()

# Funciones auxiliares
def format_stats_detailed(stats):
    return f"""📊 **ESTADO DEL SISTEMA**

**📚 DOCUMENTOS:**
• PDFs procesados: {stats.get('total_pdfs', 0)}
• Páginas totales: {stats.get('total_pages', 0):,}
• Chunks indexados: {stats.get('total_chunks', 0):,}

**⚙️ HARDWARE:**
• GPU: {'✅ NVIDIA ' + torch.cuda.get_device_name(0) if torch.cuda.is_available() else '❌ CPU'}
• Memoria GPU: {torch.cuda.memory_allocated()/1e9:.1f}GB / {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB

**🔄 ÚLTIMA ACTUALIZACIÓN:**
{stats.get('last_update', '')}"""

def get_system_stats():
    stats = pdf_rag.get_system_info()
    return format_stats_detailed(stats)

# ===== INTERFAZ PRINCIPAL =====
with gr.Blocks(title="RAG Hispanidad", theme=gr.themes.Soft()) as simple_demo:

    # Estado - formato para Gradio 6+: lista de listas
    chat_history = gr.State([])

    # Título
    gr.Markdown("## 🤖 **RAG Hispanidad - Chat con PDFs Históricos**")
    gr.Markdown("Sube PDFs históricos y conversa con ellos usando inteligencia artificial")

    # Chatbot SIN avatar_images (causa error en Gradio 6+)
    chatbot = gr.Chatbot(
        label="💬 Conversación",
        height=450,
        bubble_full_width=False
    )

    # Área de entrada
    user_input = gr.Textbox(
        label="Tu pregunta sobre historia hispánica",
        placeholder="Ej: ¿Qué documentos tienes sobre la Leyenda Negra española?",
        lines=3,
        max_lines=5
    )

    # Botones principales
    with gr.Row():
        submit_btn = gr.Button("📤 Enviar pregunta", variant="primary", size="lg")
        clear_btn = gr.Button("🗑️ Limpiar chat", variant="secondary")
        test_btn = gr.Button("🧪 Probar sistema", variant="secondary")

    # Panel izquierdo: Gestión de documentos
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📄 **Gestión de PDFs**")

            pdf_upload = gr.File(
                label="Arrastra o selecciona PDFs históricos",
                file_types=[".pdf"],
                file_count="multiple",
                height=100
            )

            with gr.Row():
                pdf_process_btn = gr.Button("🔧 Procesar PDFs", variant="primary")
                pdf_clear_btn = gr.Button("Limpiar", variant="secondary")

            pdf_status = gr.Textbox(
                label="Estado de procesamiento",
                value="Listo para recibir PDFs...",
                interactive=False,
                lines=3
            )

        # Panel derecho: Sistema y estadísticas
        with gr.Column(scale=1):
            gr.Markdown("### 📊 **Estado del Sistema**")

            stats_display = gr.Textbox(
                label="Estadísticas en tiempo real",
                value="Calculando...",
                interactive=False,
                lines=6
            )

            gr.Markdown("### ⚙️ **Configuración**")

            response_length = gr.Slider(
                minimum=500,
                maximum=5000,
                value=2000,
                step=100,
                label="📏 Longitud de respuesta"
            )

            refresh_btn = gr.Button("🔄 Actualizar estadísticas", variant="secondary")

    # Información del sistema
    gr.Markdown("---")
    gr.Markdown(f"""
    ### 🏛️ **Sistema RAG Hispanidad**
    - **Modelo:** {MODEL_NAME}
    - **Embeddings:** {EMBEDDING_MODEL}
    - **Base de datos:** ChromaDB persistente en Google Drive
    - **GPU:** {'✅ Disponible' if torch.cuda.is_available() else '❌ Solo CPU'}

    ### 🎯 **Cómo usar:**
    1. **Sube PDFs** históricos usando el panel izquierdo
    2. **Haz clic en "Procesar PDFs"** para indexarlos
    3. **Pregunta** sobre cualquier tema histórico
    4. **Recibe respuestas** fundamentadas en tus documentos
    """)

    # ===== FUNCIONES CORREGIDAS (GRADIO 6+) =====

    def chat_function(message, history, max_chars):
        """Función principal del chat - FORMATEADA PARA GRADIO 6+"""
        print(f"\n{'='*60}")
        print(f"🔔 CHAT LLAMADO: '{message[:80]}...'")

        try:
            # 1. Buscar documentos relevantes
            print("   🔍 Buscando en documentos...")
            docs = pdf_rag.search_documents(message, n_results=3)
            print(f"   📚 Documentos encontrados: {len(docs)}")

            # 2. Generar respuesta
            if docs:
                print(f"   🤖 Generando respuesta ({max_chars} caracteres máx)...")
                response = pdf_rag.generate_response(message, docs, max_chars)
                print(f"   ✅ Respuesta generada: {len(response):,} caracteres")
            else:
                response = """📭 **No encontré documentos relevantes en la base de datos.**

💡 **Sugerencias:**
1. Sube PDFs históricos usando el panel izquierdo
2. Haz clic en "Procesar PDFs" para indexarlos
3. Reformula tu pregunta usando términos históricos"""
                print("   ⚠️  No se encontraron documentos")

            # 3. CORRECCIÓN CLAVE: Formato para Gradio 6+
            # Usar lista de listas [mensaje_usuario, respuesta_asistente]
            history.append([message, response])

            print(f"   📊 Historial actualizado: {len(history)} intercambios")
            print(f"{'='*60}")

            return history, ""

        except Exception as e:
            print(f"❌ ERROR en chat_function: {type(e).__name__}: {e}")

            error_msg = f"""⚠️ **Error en el sistema**

**Detalles:** {str(e)[:150]}"""

            history.append([message, error_msg])
            print(f"{'='*60}")
            return history, ""

    def process_pdfs_function(files):
        """Procesar PDFs - maneja múltiples archivos"""
        if not files:
            return "❌ No se seleccionaron archivos", get_system_stats()

        print(f"\n{'='*60}")
        print(f"📤 PROCESANDO {len(files)} PDFs...")

        results = []
        total_chunks = 0

        for i, file in enumerate(files):
            print(f"   [{i+1}/{len(files)}] Procesando...")

            result = pdf_rag.upload_and_process_pdf(file)

            if result.get('success', False):
                chunks = result.get('chunks_added', 0)
                total_chunks += chunks
                results.append(f"✅ {result.get('filename', 'PDF')}: {chunks} chunks")
            else:
                results.append(f"❌ {result.get('filename', 'PDF')}: {result.get('error', 'Error')}")

        # Actualizar estadísticas
        pdf_rag.update_stats()
        stats = pdf_rag.get_system_info()

        # Crear resumen
        if total_chunks > 0:
            summary = f"✅ {len(files)} PDFs procesados, {total_chunks} chunks añadidos"
        else:
            summary = f"⚠️  {len(files)} PDFs procesados, 0 chunks añadidos"

        print(f"📊 RESUMEN: {summary}")
        print(f"{'='*60}")

        result_text = f"**Resultados del procesamiento:**\n\n" + "\n".join(results[:10])
        if len(results) > 10:
            result_text += f"\n\n... y {len(results) - 10} más"

        return result_text, format_stats_detailed(stats)

    def test_system_function(message, history):
        """Función de prueba del sistema"""
        print(f"\n🧪 PRUEBA DEL SISTEMA: '{message}'")

        test_response = f"""🧪 **Prueba del sistema completada**

✅ **Componentes verificados:**
• Modelo salamandra-2b: {'🟢 Operativo' if pdf_rag.model else '🔴 No disponible'}
• Base de vectores: {pdf_rag.vector_store.get_stats().get('total_chunks', 0):,} chunks
• Embeddings: {'🟢 Operativo' if pdf_rag.embedder else '🔴 No disponible'}
• GPU: {'🟢 Disponible' if torch.cuda.is_available() else '🟡 Solo CPU'}

📊 **Estadísticas actuales:**
{pdf_rag.get_system_info().get('total_pdfs', 0)} PDFs procesados
{pdf_rag.get_system_info().get('total_chunks', 0):,} chunks indexados

💡 **Sistema listo para usar.**"""

        history.append([message, test_response])
        return history, ""

    def clear_chat_function():
        """Limpiar el chat"""
        print("🗑️  Chat limpiado")
        return []

    def clear_pdfs_function():
        """Limpiar la lista de PDFs"""
        print("🗑️  Lista de PDFs limpiada")
        return None

    # ===== CONEXIONES CORREGIDAS =====

    # 1. CHAT PRINCIPAL
    submit_btn.click(
        fn=chat_function,
        inputs=[user_input, chat_history, response_length],
        outputs=[chatbot, user_input]
    )

    user_input.submit(
        fn=chat_function,
        inputs=[user_input, chat_history, response_length],
        outputs=[chatbot, user_input]
    )

    # 2. BOTÓN DE PRUEBA
    test_btn.click(
        fn=test_system_function,
        inputs=[user_input, chat_history],
        outputs=[chatbot, user_input]
    )

    # 3. PROCESAMIENTO DE PDFs
    pdf_process_btn.click(
        fn=process_pdfs_function,
        inputs=[pdf_upload],
        outputs=[pdf_status, stats_display]
    )

    # 4. BOTONES DE LIMPIEZA
    clear_btn.click(
        fn=clear_chat_function,
        outputs=[chatbot]
    )

    pdf_clear_btn.click(
        fn=clear_pdfs_function,
        outputs=[pdf_upload]
    )

    # 5. ACTUALIZACIÓN DE ESTADÍSTICAS
    refresh_btn.click(
        fn=get_system_stats,
        outputs=[stats_display]
    )

    # 6. CARGA INICIAL
    simple_demo.load(
        fn=get_system_stats,
        outputs=[stats_display]
    )

print("✅ Interfaz 'simple_demo' creada exitosamente!")
print(f"📊 Sistema: {pdf_rag.get_system_info().get('total_pdfs', 0)} PDFs, {pdf_rag.get_system_info().get('total_chunks', 0):,} chunks")


📤 INICIALIZANDO SISTEMA RAG...

🏛️  INICIALIZANDO SISTEMA RAG CON PDFs
📄 Extractor de PDFs inicializado


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


🆕 Nueva colección creada
🧠 Cargando modelo salamandra-2b...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.81M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


✅ SISTEMA LISTO
📚 PDFs: 0
📝 Chunks: 0
✅ Interfaz 'simple_demo' creada exitosamente!
📊 Sistema: 0 PDFs, 0 chunks


In [ ]:
# ==================== 10. LANZAR LA APLICACIÓN (VERSIÓN CORREGIDA) ====================
print("\n" + "="*70)
print("🚀 LANZANDO APLICACIÓN RAG HISPANIDAD")
print("="*70)

print(f"📊 Sistema inicializado:")
print(f"   • PDFs procesados: {pdf_rag.get_system_info().get('total_pdfs', 0)}")
print(f"   • Chunks indexados: {pdf_rag.get_system_info().get('total_chunks', 0):,}")
print(f"   • GPU: {'✅ ' + torch.cuda.get_device_name(0) if torch.cuda.is_available() else '❌ CPU'}")

print("\n🎯 **INSTRUCCIONES FINALES:**")
print("1. Sube PDFs históricos usando el panel izquierdo")
print("2. Haz clic en '🔧 Procesar PDFs' para indexarlos")
print("3. Pregunta sobre cualquier tema histórico")
print("4. ¡Todo se guarda automáticamente en tu Google Drive!")

print("\n🌐 Abriendo interfaz web...")
print("   La URL pública estará disponible en unos segundos")

# Lanzar la aplicación con configuración óptima para Colab
try:
    # Intenta con share=True para URL pública
    simple_demo.launch(
        share=True,
        debug=False  # debug=False para menos ruido en Colab
    )
except Exception as e:
    print(f"⚠️ No se pudo crear URL pública: {e}")
    print("📱 Usando enlace local...")
    # Fallback a solo local
    simple_demo.launch(
        share=False
    )


🚀 LANZANDO APLICACIÓN RAG HISPANIDAD
📊 Sistema inicializado:
   • PDFs procesados: 0
   • Chunks indexados: 0
   • GPU: ✅ Tesla T4

🎯 **INSTRUCCIONES FINALES:**
1. Sube PDFs históricos usando el panel izquierdo
2. Haz clic en '🔧 Procesar PDFs' para indexarlos
3. Pregunta sobre cualquier tema histórico
4. ¡Todo se guarda automáticamente en tu Google Drive!

🌐 Abriendo interfaz web...
   La URL pública estará disponible en unos segundos
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://468754c724fb7b43d7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
